In [ ]:
!pip install transformers datasets

In [ ]:
from psutil import virtual_memory
from google.colab import files
import pandas as pd
import numpy as np
import io
import nltk
import re
import string
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import DataCollatorWithPadding
from transformers import TFAutoModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import Dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
uploaded = files.upload()

Saving homomex_training.csv to homomex_training.csv


In [ ]:
task1_df = pd.read_csv(io.BytesIO(uploaded['homomex_training.csv']))

In [ ]:
le = LabelEncoder()
le.fit(task1_df['label'])

LabelEncoder()

In [ ]:
list(le.classes_)

['NP', 'P', nan]

In [ ]:
SEED = 2

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(list(task1_df['tweets']), list(le.transform(task1_df['label'])),
                                                                    test_size=.2, random_state=SEED, shuffle=True)

In [ ]:
print(len(train_texts))
print(len(val_texts))

5600
1400


In [ ]:
checkpoint = "dccuchile/bert-base-spanish-wwm-cased" 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
batch_size = 4
num_epochs = 4

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

/usr/local/lib/python3.9/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler)
loss=SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3,
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.1,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/4
1400/1400 [==============================] - 172s 88ms/step - loss: 0.5071 - accuracy: 0.8052 - val_loss: 0.4759 - val_accuracy: 0.8136
Epoch 2/4
1400/1400 [==============================] - 101s 72ms/step - loss: 0.3516 - accuracy: 0.8718 - val_loss: 0.4041 - val_accuracy: 0.8607
Epoch 3/4
1400/1400 [==============================] - 101s 72ms/step - loss: 0.1978 - accuracy: 0.9341 - val_loss: 0.4465 - val_accuracy: 0.8650
Epoch 4/4
1400/1400 [==============================] - 100s 72ms/step - loss: 0.0804 - accuracy: 0.9789 - val_loss: 0.5251 - val_accuracy: 0.8600


In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)

350/350 [==============================] - 12s 27ms/step


In [ ]:
print(classification_report(class_preds, val_encodings['labels']))

              precision    recall  f1-score   support

           0       0.92      0.89      0.90       899
           1       0.57      0.65      0.61       138
           2       0.85      0.87      0.86       363

    accuracy                           0.86      1400
   macro avg       0.78      0.80      0.79      1400
weighted avg       0.87      0.86      0.86      1400

